In [5]:
import ipywidgets as widgets

In [6]:
slider = widgets.FloatSlider(
    value=7.5,
    min=5.0,
    max=10.0,
    step=0.1,
    description='Input:',
)

slider

FloatSlider(value=7.5, description='Input:', max=10.0, min=5.0)

In [9]:
!ls

data_extractions.ipynb widget_vis.ipynb


In [7]:
import json
import os
import pandas as pd

In [8]:
f = open('/Users/vaibhavjade/classwork/data science/hockey-primer-1/data/trial.json')

In [9]:
sample = json.load(f)

sample

In [10]:
sample.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [29]:
sample['gameData']

{'game': {'pk': 2017020021, 'season': '20172018', 'type': 'R'},
 'datetime': {'dateTime': '2017-10-07T23:00:00Z',
  'endDateTime': '2017-10-08T01:47:01Z'},
 'status': {'abstractGameState': 'Final',
  'codedGameState': '7',
  'detailedState': 'Final',
  'statusCode': '7',
  'startTimeTBD': False},
 'teams': {'away': {'id': 18,
   'name': 'Nashville Predators',
   'link': '/api/v1/teams/18',
   'venue': {'id': 5030,
    'name': 'Bridgestone Arena',
    'link': '/api/v1/venues/5030',
    'city': 'Nashville',
    'timeZone': {'id': 'America/Chicago', 'offset': -5, 'tz': 'CDT'}},
   'abbreviation': 'NSH',
   'triCode': 'NSH',
   'teamName': 'Predators',
   'locationName': 'Nashville',
   'firstYearOfPlay': '1998',
   'division': {'id': 16, 'name': 'Central', 'link': '/api/v1/divisions/16'},
   'conference': {'id': 5, 'name': 'Western', 'link': '/api/v1/conferences/5'},
   'franchise': {'franchiseId': 34,
    'teamName': 'Predators',
    'link': '/api/v1/franchises/34'},
   'shortName': 'Nas

In [32]:
sample['liveData']['plays']['allPlays'][6]

{'players': [{'player': {'id': 8471215,
    'fullName': 'Evgeni Malkin',
    'link': '/api/v1/people/8471215'},
   'playerType': 'Scorer',
   'seasonTotal': 1},
  {'player': {'id': 8473548,
    'fullName': 'Phil Kessel',
    'link': '/api/v1/people/8473548'},
   'playerType': 'Assist',
   'seasonTotal': 1},
  {'player': {'id': 8474176,
    'fullName': 'Carl Hagelin',
    'link': '/api/v1/people/8474176'},
   'playerType': 'Assist',
   'seasonTotal': 1},
  {'player': {'id': 8477424,
    'fullName': 'Juuse Saros',
    'link': '/api/v1/people/8477424'},
   'playerType': 'Goalie'}],
 'result': {'event': 'Goal',
  'eventCode': 'PIT56',
  'eventTypeId': 'GOAL',
  'description': 'Evgeni Malkin (1) Snap Shot, assists: Phil Kessel (1), Carl Hagelin (1)',
  'secondaryType': 'Snap Shot',
  'strength': {'code': 'EVEN', 'name': 'Even'},
  'gameWinningGoal': True,
  'emptyNet': False},
 'about': {'eventIdx': 6,
  'eventId': 56,
  'period': 1,
  'periodType': 'REGULAR',
  'ordinalNum': '1st',
  'peri

In [33]:
for i in sample['liveData']['plays']['allPlays']:
    print(i.keys())

dict_keys(['result', 'about', 'coordinates'])
dict_keys(['result', 'about', 'coordinates'])
dict_keys(['result', 'about', 'coordinates'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['players', 'result', 'about', 'coordinates', 'team'])
dict_keys(['playe

In [24]:
df = pd.DataFrame.from_dict(sample['liveData']['plays']['allPlays'])

In [26]:
df

,result,about,coordinates,players,team
0,"{'event': 'Game Scheduled', 'eventCode': 'PIT1...","{'eventIdx': 0, 'eventId': 1, 'period': 1, 'pe...",{},NaN,NaN
1,"{'event': 'Period Ready', 'eventCode': 'PIT4',...","{'eventIdx': 1, 'eventId': 4, 'period': 1, 'pe...",{},NaN,NaN
2,"{'event': 'Period Start', 'eventCode': 'PIT52'...","{'eventIdx': 2, 'eventId': 52, 'period': 1, 'p...",{},NaN,NaN
3,"{'event': 'Faceoff', 'eventCode': 'PIT53', 'ev...","{'eventIdx': 3, 'eventId': 53, 'period': 1, 'p...","{'x': 0.0, 'y': 0.0}","[{'player': {'id': 8471675, 'fullName': 'Sidne...","{'id': 5, 'name': 'Pittsburgh Penguins', 'link..."
4,"{'event': 'Missed Shot', 'eventCode': 'PIT54',...","{'eventIdx': 4, 'eventId': 54, 'period': 1, 'p...","{'x': -69.0, 'y': 9.0}","[{'player': {'id': 8474009, 'fullName': 'Nick ...","{'id': 18, 'name': 'Nashville Predators', 'lin..."
...,...,...,...,...,...
326,"{'event': 'Missed Shot', 'eventCode': 'PIT681'...","{'eventIdx': 326, 'eventId': 681, 'period': 3,...","{'x': 73.0, 'y': -6.0}","[{'player': {'id': 8475810, 'fullName': 'Bryan...","{'id': 5, 'name': 'Pittsburgh Penguins', 'link..."
327,"{'event': 'Hit', 'eventCode': 'PIT739', 'event...","{'eventIdx': 327, 'eventId': 739, 'period': 3,...","{'x': 83.0, 'y': 41.0}","[{'player': {'id': 8475832, 'fullName': 'Tom K...","{'id': 5, 'name': 'Pittsburgh Penguins', 'link..."
328,"{'event': 'Period End', 'eventCode': 'PIT740',...","{'eventIdx': 328, 'eventId': 740, 'period': 3,...",{},NaN,NaN
329,"{'event': 'Period Official', 'eventCode': 'PIT...","{'eventIdx': 329, 'eventId': 743, 'period': 3,...",{},NaN,NaN
